# Challenge Alura - Reduzir a taxa de evasão dos clientes (Churn Rate)
## Analisar os clientes para identificar um padrão que determine aqueles que possuem maior chance de deixar de consumir o serviço, ou deixar de ser cliente da empresa

## Para iniciar os estudos, inicia-se a importação das bibliotecas básicas para tal e o dataset a ser analisado, entendendo que o primeiro passo é entender os dados a serem utilizados.

### Importando bibliotecas

Para iniciar uma análise exploratória é necessário transformar e visualizar os dados.

Foram selecionadas duas bibliotecas muito comuns e de bom desempenho para isto, o pandas e numpy.

Posteriormente serão importadas outras bibliotecas, principalmente quando forem geradas visualizações gráficas, mas isso pode aguardar.

In [1]:
import pandas as pd
import numpy as np

### Importando o dataset e tendo a primeira visualização

Como a base dados a ser estudada está em formato json, optei por utilizar o pd.read_json() para fazer a leitura, principalmente porque apresenta os dados em formato tabular, facilitando as primeiras impressões e possibilitando nortear os próximos passos.

In [2]:
df = pd.read_json('https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json')
display(df.head(2))

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Os dados importados geraram uma tabela, porém, a mesma apresenta colunas ainda em formato json, dificultando o entendimento e impossibilitando análises estatísticas básicas de maneira simples.

Sendo assim, será necessário iniciar a transformação dos dados.

### Transformando os dados de json para dataframe pandas.

Todas as colunas a partir da Churn estavam em formato json ainda, dificultando a visualização completa bem como cálculos e análises a serem realizados.

Foi efetuada a conversão dos dados, primeiramente, criando dataframes separados para cada coluna a ser tratada (customer, phone, internet e account), utilizando a função pd.json_normalize().

Na coluna account foi necessário uma atenção maior, já que havia uma cadeia de json (Charges), situação que foi tratada pela função pd.json_normalize().

In [3]:
# Separando valores que estão em formato json, e retornando para o dataframe
customer = pd.json_normalize(df.customer)
phone = pd.json_normalize(df.phone)
internet = pd.json_normalize(df.internet)
account = pd.json_normalize(df.account)
df_base = pd.concat([df.customerID, df.Churn, customer, phone, internet, account], axis = 1)

### Fazendo a tradução dos cabeçalhos

A base de dados está toda em inglês, e para facilitar o entendimento do que cada coluna significa, foi efetuada a tradução das mesmas.

Posteriormente será analisada a necessidade de traduzir também os outros dados do dataset.

In [4]:
# Traduzindo nomes das colunas manualmente
df_base.columns = ['clienteID', 
                   'Churn', 
                   'genero_cliente', 
                   'mais_que_65_anos', 
                   'possui_parceiro_a', 
                  'possui_dependentes', 
                  'meses_contrato',
                  'servico_telefonico',
                  'multiplas_linhas',
                  'servico_internet',
                  'assinatura_seguranca',
                  'assinatura_backup',
                  'assinatura_protecao_dispositivo',
                  'assinatura_suporte',
                  'assinatura_TV',
                  'assinatura_streaming',
                  'contrato',
                  'fatura_digital',
                  'forma_pagamento',
                  'total_servicos_mes',
                  'total_gasto']

df_base.head()

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Agora que foi possível entender do que se trata cada coluna é importante verificar se os dados existentes nelas são consistentes e possibilitam efetuar análises.

Primeiramente é necessário verificar se há dados faltantes, e se eles impossibilitarão análises posterior.

O pandas tem a função .info() que retorna os tipos de cada coluna, além de informar se há dados nulos nelas.

In [27]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   clienteID                        7267 non-null   object 
 1   Churn                            7267 non-null   object 
 2   genero_cliente                   7267 non-null   object 
 3   mais_que_65_anos                 7267 non-null   int64  
 4   possui_parceiro_a                7267 non-null   object 
 5   possui_dependentes               7267 non-null   object 
 6   meses_contrato                   7267 non-null   int64  
 7   servico_telefonico               7267 non-null   object 
 8   multiplas_linhas                 7267 non-null   object 
 9   servico_internet                 7267 non-null   object 
 10  assinatura_seguranca             7267 non-null   object 
 11  assinatura_backup                7267 non-null   object 
 12  assinatura_protecao_

A coluna total_gasto apresenta o tipo 'object', porém, se trata de um valor numérico flutuante, ou float, e por isto se faz necessária a conversão do mesmo.

Utilizei a função pd.to_numeric para isto.

In [28]:
# Convertendo coluna total_gasto para float
df_base.total_gasto = pd.to_numeric(df_base.total_gasto, errors='coerce')
df_base.head(2)

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4


Após converter os dados, importante verificar se houve a geração de algum NA, pois estes podem atrapalhar a análise e montagem de gráficos.

In [31]:
df_base[df_base.total_gasto.isna()]

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


Foram identificadas 11 linhas em que o total_gasto contém NaN, coincidindo com as linhas em que os clientes ainda não possuem um mês completo de utilização dos serviços (meses_contrato).

Será considerado o valor da coluna total_servicos_mes nestes casos, pois indica o valor gasto pelo cliente naquele período, mesmo que incompleto.

Achei interessante a ideia de criar uma função para validar os valores apresentados nas colunas para evitar a repetição de código.

In [ ]:
def valores_unicos():
    for coluna in df_base.loc[:,'Churn':].columns:
        if len(df_base[f'{coluna}'].unique()) > 5:
            return print(f'{coluna} contém muitos valores únicos, analisar com maior detalhamento.')
        else:
            valores = df_base[f'{coluna}'].unique()
            return print(f'{coluna}: ' + str(valores))


In [ ]:
valores_unicos()

In [ ]:
# Excluindo linhas com dados vazios na coluna Churn
df_base.drop(df_base[df_base.Churn == ''].index, inplace = True)
df_base.reset_index()

# Substituindo valores vazios por 0 na coluna total_gasto
df_base.total_gasto .replace(' ', '', inplace = True)



# Criando coluna gastos_diarios, quando meses_contrato < 0, usar coluna total_servicos_mes,
# quando meses_contrato > 0 considera total_gasto
df_base['gastos_diarios'] = [df_base.total_gasto[i] / (df_base.meses_contrato[i] * 30) 
                             if df_base.meses_contrato[i] > 0 
                             else df_base.total_servicos_mes[i] / 30 
                             for i in df_base.index]

df_base.head(2)

### Validação de algumas colunas

In [ ]:
df_base.columns

In [ ]:
df_base.assinatura_protecao_dispositivo.unique()

In [ ]:
df_base[df_base.multiplas_linhas == 'No phone service'].shape[0]

In [ ]:
pd.crosstab(df_base.Churn, df_base.servico_internet)

In [ ]:
df_base.groupby('servico_internet').count()

In [ ]:
df_base.groupby('assinatura_seguranca').count()

In [ ]:
df_base.groupby('assinatura_backup').count()

In [ ]:
df_base.groupby('assinatura_protecao_dispositivo').count()

### Análise da variável target - Churn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_base.Churn.value_counts()

#### Avaliando por gênero

In [ ]:
df_genero = pd.crosstab(df_base.Churn, df_base.genero_cliente)
df_genero

In [ ]:
df_base.groupby('genero_cliente')['Churn'].count()

In [ ]:
df_genero = pd.crosstab(df_base.Churn, df_base.genero_cliente)

fig, ax = plt.subplots(figsize=(10, 8))

grafico = ax.bar(x = 'Churn', height=df_base.Churn, data = df_base)

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# grafico = df_genero.plot(kind = 'bar')
grafico = ax.bar(x = df_genero.columns, height=df_genero.loc['Yes'], data = df_genero)

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)

#### Se acima de 65 anos

In [ ]:
df_analise = pd.crosstab(df_base.Churn, df_base.mais_que_65_anos.astype(str))

fig, ax = plt.subplots(figsize=(10, 8))

grafico = ax.bar(x = df_analise.columns, height=df_analise.loc['Yes'], data = df_analise.loc['Yes'])

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)